# 00E — Nowcasting: Financial Conditions Index (FCI)

**Purpose**: Create a high-frequency "Market Stress Gauge" (Nowcaster) since real economy data is lagged.

**Components**:
1.  **Cost of Capital**: 10Y G-Sec Yield (Proxies long-term funidng cost)
2.  **Banking Liquidity**: Call Money Rate (Proxies short-term stress)
3.  **External Buffer**: Forex Reserves (Proxies currency defense ammo)
4.  **Risk Sediment**: USDINR Volatility (Proxies fear)

**Output**: `../data_processed/india_fci_weekly.parquet` (Composite Z-Score)

---

## 1. Load Data

In [1]:
import pandas as pd
import numpy as np
from pathlib import Path

PROCESSED_PATH = Path('../data_processed')
RBI_PATH = PROCESSED_PATH / 'rbi_macro_all_long.parquet'
GLOBAL_PATH = PROCESSED_PATH / 'global_fx_weekly.parquet'

# Load RBI Data
if RBI_PATH.exists():
    rbi_df = pd.read_parquet(RBI_PATH)
    # Filter for HF series
    hf_series = [
        '10-Year G-Sec Yield (FBIL) (%)', 
        'CALL MONEY RATE (BORROWINGS) - HIGH',
        'FOREIGN EXCHANGE RESERVES'
    ]
    rbi_hf = rbi_df[rbi_df['series_name'].isin(hf_series)].copy()
    print(f"Loaded RBI HF Data: {len(rbi_hf)} rows")
else:
    raise FileNotFoundError("RBI data missing. Run 00B.")

# Load Global Data for USDINR
if GLOBAL_PATH.exists():
    global_fx = pd.read_parquet(GLOBAL_PATH)
    usdinr = global_fx['USDINR'].rename('USDINR_Rate')
    print(f"Loaded Global FX Data: {len(global_fx)} rows")
else:
    print("Global FX missing. Will proceed without USDINR Vol.")
    usdinr = pd.Series()

Loaded RBI HF Data: 1584 rows
Loaded Global FX Data: 840 rows


## 2. Process Components (Weekly Resample)

We standardize everything to **Weekly (Friday)** frequency.

In [2]:
# Pivot RBI data
rbi_wide = rbi_hf.pivot_table(index='Date', columns='series_name', values='value')

# Rename columns for valid variables
col_map = {
    '10-Year G-Sec Yield (FBIL) (%)': 'GSEC_10Y',
    'CALL MONEY RATE (BORROWINGS) - HIGH': 'CALL_RATE',
    'FOREIGN EXCHANGE RESERVES': 'FX_RESERVES'
}
rbi_wide = rbi_wide.rename(columns=col_map)

# Resample RBI to Weekly
fci_df = rbi_wide.resample('W-FRI').last().ffill()

# Add USDINR Volatility (if available)
if not usdinr.empty:
    # Calculate 4-week Rolling Volatility of USDINR
    usdinr_weekly = usdinr.resample('W-FRI').last().ffill()
    usdinr_vol = usdinr_weekly.pct_change().rolling(4).std() * 100
    fci_df['USDINR_VOL_4W'] = usdinr_vol

print("FCI Components Matrix:")
print(fci_df.tail())

FCI Components Matrix:
series_name  GSEC_10Y  CALL_RATE  FX_RESERVES  USDINR_VOL_4W
Date                                                        
2025-12-26       6.65       5.65  6258847.772       0.341339
2026-01-02       6.69       6.00  6194458.394       0.296204
2026-01-09       6.71       5.80  6195895.769       0.081458
2026-01-16       6.75       5.80  6373301.584       0.321983
2026-01-23       6.74       5.80  6523644.584       0.681542


## 3. Calculate Z-Scores (Standardize)

Each component has different units. We use **Rolling 3-Year Z-Scores**.

**Directionality for Stress (Higher Z = TIGHTER/WORSE)**:
- **GSEC_10Y**: Higher = Tighter (Bad) -> **+Z**
- **CALL_RATE**: Higher = Tighter (Bad) -> **+Z**
- **FX_RESERVES**: Lower = Tighter (Bad) -> **-Z** (Need to invert)
- **USDINR_VOL**: Higher = Volatile (Bad) -> **+Z**

In [3]:
def rolling_z(series, window=156): # 3 years * 52 weeks
    return (series - series.rolling(window, min_periods=26).mean()) / series.rolling(window, min_periods=26).std()

fci_z = pd.DataFrame(index=fci_df.index)

if 'GSEC_10Y' in fci_df.columns:
    fci_z['GSEC_Z'] = rolling_z(fci_df['GSEC_10Y'])

if 'CALL_RATE' in fci_df.columns:
    fci_z['CALL_Z'] = rolling_z(fci_df['CALL_RATE'])

if 'FX_RESERVES' in fci_df.columns:
    # Invert (Lower Reserves = Higher Stress)
    fci_z['FX_Z'] = rolling_z(fci_df['FX_RESERVES']) * -1

if 'USDINR_VOL_4W' in fci_df.columns:
    fci_z['FX_VOL_Z'] = rolling_z(fci_df['USDINR_VOL_4W'])

# Composite FCI (Equal Weight Average of available Z-scores)
fci_z['FCI_COMPOSITE_Z'] = fci_z.mean(axis=1)

# Smooth the final index (4-week moving average) to reduce noise
fci_z['FCI_SMOOTH'] = fci_z['FCI_COMPOSITE_Z'].rolling(4).mean()

print("FCI Z-Scores:")
print(fci_z.tail())

FCI Z-Scores:
              GSEC_Z    CALL_Z      FX_Z  FX_VOL_Z  FCI_COMPOSITE_Z  \
Date                                                                  
2025-12-26 -0.795071 -1.708426 -1.722632 -0.298739        -1.131217   
2026-01-02 -0.660974 -1.074195 -1.569733 -0.454611        -0.939878   
2026-01-09 -0.588732 -1.418794 -1.553296 -1.195775        -1.189149   
2026-01-16 -0.453519 -1.401902 -1.891841 -0.337018        -1.021070   
2026-01-23 -0.475458 -1.384133 -2.161608  0.960293        -0.765227   

            FCI_SMOOTH  
Date                    
2025-12-26   -1.175394  
2026-01-02   -1.109503  
2026-01-09   -1.090105  
2026-01-16   -1.070329  
2026-01-23   -0.978831  


## 4. Export

In [4]:
if not fci_z.empty:
    output_path = PROCESSED_PATH / 'india_fci_weekly.parquet'
    fci_z.to_parquet(output_path)
    print(f"✓ Saved: {output_path}")
else:
    print("No FCI data generated.")

✓ Saved: ..\data_processed\india_fci_weekly.parquet
